## Explore training data

<a target="_blank" href="https://colab.research.google.com/github/Pappa/bandido/blob/main/notebooks/crypto/01b-eda.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
import pandas as pd
from IPython.display import display

pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [5]:
df = pd.read_csv('data/ohlcv.csv.gz', compression='gzip')
df.head(5)

,timestamp,symbol,open,high,low,close,volume
0,2024-01-01 00:00:00,BTC/USDT,42283.5800,42554.5700,42261.0200,42475.2300,1271.6811
1,2024-01-01 00:00:00,XRP/USDT,0.6155,0.6172,0.6146,0.6162,14498728.0000
2,2024-01-01 00:00:00,DOGE/USDT,0.0896,0.0900,0.0895,0.0898,17799677.0000
3,2024-01-01 00:00:00,SOL/USDT,101.7200,102.7900,101.5600,101.9600,196680.9300
4,2024-01-01 00:00:00,ETH/USDT,2281.8700,2297.1800,2281.2700,2295.5100,10771.9183


In [ ]:
from trading_utils import preprocess_data, create_wide_format_data, SYMBOLS

all_data = preprocess_data(df)
observations_df, prices_df = create_wide_format_data(
    all_data, 
    symbols=SYMBOLS, 
    features=['rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_percent_b', 'bb_bandwidth']
)

In [ ]:
# observation data
observations_df.head(5)

,BTC_rsi,BTC_macd,BTC_macd_signal,BTC_macd_hist,BTC_bb_percent_b,BTC_bb_bandwidth,DOGE_rsi,DOGE_macd,DOGE_macd_signal,DOGE_macd_hist,...,ETH_macd_signal,ETH_macd_hist,ETH_bb_percent_b,ETH_bb_bandwidth,SOL_rsi,SOL_macd,SOL_macd_signal,SOL_macd_hist,SOL_bb_percent_b,SOL_bb_bandwidth
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 19:00:00,86.4854,160.4117,79.4254,80.9863,1.2714,0.0263,86.8952,0.0004,0.0002,0.0002,...,3.7600,4.3128,1.0745,0.0286,82.8372,1.4074,0.9161,0.4913,1.0453,0.0777
2024-01-01 20:00:00,87.5550,215.2048,106.5813,108.6235,1.1769,0.0337,79.9977,0.0005,0.0003,0.0002,...,4.9768,4.8672,1.0207,0.0328,73.9572,1.4614,1.0252,0.4363,0.8831,0.0797
2024-01-01 21:00:00,84.6400,250.2050,135.3060,114.8989,1.0195,0.0386,81.3002,0.0005,0.0003,0.0002,...,6.1551,4.7131,0.9194,0.0356,74.3974,1.4998,1.1201,0.3797,0.8574,0.0843
2024-01-01 22:00:00,80.8682,267.6534,161.7755,105.8779,0.9057,0.0415,75.0780,0.0005,0.0004,0.0001,...,7.1756,4.0823,0.8347,0.0373,74.5956,1.5169,1.1995,0.3174,0.8246,0.0876
2024-01-01 23:00:00,85.8193,330.0952,195.4394,134.6557,1.1094,0.0489,78.4287,0.0005,0.0004,0.0001,...,8.3120,4.5454,0.9740,0.0390,79.7231,1.6581,1.2912,0.3669,0.9494,0.0943


In [8]:
# prices data
prices_df.head(5)

symbol,BTC,DOGE,XRP,ETH,SOL
timestamp,,,,,
2024-01-01 19:00:00,43517.9900,0.0918,0.6258,2337.2600,108.8100
2024-01-01 20:00:00,43687.8600,0.0915,0.6276,2340.9800,107.8800
2024-01-01 21:00:00,43619.2100,0.0917,0.6282,2337.8300,108.0400
2024-01-01 22:00:00,43529.9300,0.0915,0.6269,2334.2100,108.0900
2024-01-01 23:00:00,44179.5500,0.0920,0.6294,2352.0400,109.9100
